<a href="https://colab.research.google.com/github/Fa-Abdullah/arabic_voice_assistant-clinic/blob/main/voice_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit sounddevice openai-whisper pyttsx3 requests reportlab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=3185b9cec06368fb765d9289bec6ad62f0ea260404c254eea090299cfbc2d3c2
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [3]:
!pip install sounddevice


In [4]:
!apt-get update
!apt-get install portaudio19-dev -y

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,799 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,311 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://arch

In [5]:
import streamlit as st
import sounddevice as sd
import numpy as np
import whisper
import requests
import pyttsx3
from reportlab.pdfgen import canvas
from io import BytesIO

# ========= إعدادات =========
OPENROUTER_KEY = "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"   # حطي المفتاح اللي جايبـاه من OpenRouter
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"

# تحميل موديل Whisper
@st.cache_resource
def load_whisper_model():
    return whisper.load_model("base")  # ممكن تغيري لـ "small" أو "medium" حسب الإمكانيات

model = load_whisper_model()

# ========= تسجيل الصوت =========
def record_audio(duration=5, fs=16000):
    st.info("📢 جاري التسجيل...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype="float32")
    sd.wait()
    st.success("✅ تم التسجيل")
    return np.squeeze(audio)

# ========= تحويل الصوت لنص =========
def transcribe_audio(audio, fs=16000):
    # Whisper محتاج ملف
    temp_file = "temp.wav"
    import soundfile as sf
    sf.write(temp_file, audio, fs)
    result = model.transcribe(temp_file, language="ar")
    return result["text"]

# ========= استدعاء OpenRouter =========
def chat_with_openrouter(prompt):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "gpt-4o-mini",  # ممكن تغيري الموديل
        "messages": [
            {"role": "system", "content": "أنت مساعد ذكي بالعربية"},
            {"role": "user", "content": prompt}
        ]
    }
    r = requests.post(OPENROUTER_URL, headers=headers, json=payload)
    if r.status_code == 200:
        data = r.json()
        return data["choices"][0]["message"]["content"]
    else:
        return f"خطأ: {r.text}"

# ========= تحويل نص لصوت =========
def speak_text(text):
    engine = pyttsx3.init()
    engine.setProperty("rate", 160)   # سرعة
    engine.setProperty("voice", "arabic")  # حسب الأصوات المتاحة
    engine.say(text)
    engine.runAndWait()

# ========= حفظ الرد في ملف PDF =========
def save_pdf(text):
    buffer = BytesIO()
    c = canvas.Canvas(buffer)
    c.drawString(100, 800, "رد الشات بوت:")
    c.drawString(100, 780, text)
    c.save()
    buffer.seek(0)
    return buffer

# ========= واجهة Streamlit =========
st.title("🤖 شات بوت صوتي بالعربية (بايثون فقط)")

option = st.radio("اختيار الإدخال:", ["🎤 تسجيل صوت", "⌨️ كتابة نص"])

if option == "🎤 تسجيل صوت":
    duration = st.slider("مدة التسجيل بالثواني:", 3, 15, 5)
    if st.button("ابدأ التسجيل"):
        audio = record_audio(duration)
        text_input = transcribe_audio(audio)
        st.text_area("النص المستخرج:", text_input, height=100)
        if st.button("إرسال إلى الشات بوت"):
            reply = chat_with_openrouter(text_input)
            st.text_area("رد البوت:", reply, height=150)
            if st.button("تشغيل الرد صوتيًا"):
                speak_text(reply)
            if st.button("تحميل الرد PDF"):
                pdf_buf = save_pdf(reply)
                st.download_button("تحميل PDF", pdf_buf, "reply.pdf")

else:
    user_text = st.text_area("اكتب رسالتك هنا:", height=100)
    if st.button("إرسال إلى الشات بوت"):
        reply = chat_with_openrouter(user_text)
        st.text_area("رد البوت:", reply, height=150)
        if st.button("تشغيل الرد صوتيًا"):
            speak_text(reply)
        if st.button("تحميل الرد PDF"):
            pdf_buf = save_pdf(reply)
            st.download_button("تحميل PDF", pdf_buf, "reply.pdf")


2025-09-17 10:50:50.472 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:50:50.640 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-17 10:50:50.642 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:50:50.645 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:50:50.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
 10%|███▉                                  | 14.5M/139M [00:00<00:02, 46.2MiB/s]2025-09-17 10:50:51.159 Thread 'Thread-3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:50:51.160 Thread 'Thread-3': missing ScriptRunContext! This warning can be ignore

In [7]:
"""
Arabic Voice Assistant - Online Deployment Version
Optimized for Streamlit Cloud, Hugging Face Spaces, and Google Colab
"""

import streamlit as st
import numpy as np
import tempfile
import os
import base64
import json
import time
import io
from pathlib import Path
import requests

# Core imports with fallbacks
try:
    from gtts import gTTS
    from openai import OpenAI
    import speech_recognition as sr
    from pydub import AudioSegment
except ImportError as e:
    st.error(f"Missing package: {e}")
    st.stop()

# Page configuration
st.set_page_config(
    page_title="🦷 عيادة فانكوفر لطب الأسنان",
    page_icon="🦷",
    layout="wide"
)

# Session state initialization
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []
if 'assistant_ready' not in st.session_state:
    st.session_state.assistant_ready = True  # Always ready for online version
if 'chat_mode' not in st.session_state:
    st.session_state.chat_mode = "hybrid"  # "text", "audio", "hybrid"
if 'first_message_sent' not in st.session_state:
    st.session_state.first_message_sent = False

class OnlineArabicVoiceAssistant:
    def __init__(self):
        """Initialize Online Arabic Voice Assistant"""
        # Check if running in different environments
        self.is_colab = 'google.colab' in str(get_ipython()) if 'get_ipython' in globals() else False
        self.is_local = not (os.environ.get('STREAMLIT_SHARING_MODE') or
                           os.environ.get('HUGGINGFACE_HUB_CACHE'))

        # Initialize OpenAI client
        self.openai_client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key="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"
        )

        # Initialize Speech Recognition
        self.recognizer = sr.Recognizer()

        # Arabic system prompt
        self.system_prompt = """أنت ساندي، موظفة استقبال في عيادة فانكوفر لطب الأسنان.

المعلومات المهمة:
- أوقات العمل: الاثنين إلى الجمعة من 8 صباحاً إلى 6 مساءً، السبت من 9 صباحاً إلى 3 مساءً
- الموقع: وسط مدينة فانكوفر
- الهاتف: (604) 555-DENTAL
- الخدمات: طب الأسنان العام، تنظيف الأسنان، الحشوات، التيجان، علاج الجذور، طب الأسنان التجميلي

تعليمات مهمة:
1. اجيبي باللغة العربية فقط
2. كوني ودودة ومهنية
3. اجعلي الردود قصيرة وواضحة
4. اسألي دائماً كيف يمكنك المساعدة أكثر
5. عند حجز المواعيد، اطلبي اسم المريض ونوع الخدمة المطلوبة

مثال للترحيب: "مرحباً، أهلاً وسهلاً بك في عيادة فانكوفر لطب الأسنان. اسمي ساندي، كيف يمكنني مساعدتك اليوم؟"
"""

    def transcribe_audio_google(self, audio_data, language='ar-SA'):
        """Transcribe audio using Google Speech Recognition"""
        try:
            if isinstance(audio_data, str) and audio_data.startswith('data:'):
                # Handle base64 audio data
                header, encoded = audio_data.split(',', 1)
                audio_bytes = base64.b64decode(encoded)
                audio_segment = AudioSegment.from_file(io.BytesIO(audio_bytes))
            elif isinstance(audio_data, bytes):
                audio_segment = AudioSegment.from_file(io.BytesIO(audio_data))
            else:
                # Handle file path
                audio_segment = AudioSegment.from_file(audio_data)

            # Convert to WAV format for recognition
            wav_data = audio_segment.export(format="wav").read()
            audio_source = sr.AudioFile(io.BytesIO(wav_data))

            with audio_source as source:
                self.recognizer.adjust_for_ambient_noise(source)
                audio_recorded = self.recognizer.record(source)

            # Recognize speech
            text = self.recognizer.recognize_google(
                audio_recorded,
                language=language
            )
            return text.strip()

        except sr.UnknownValueError:
            return ""
        except sr.RequestError as e:
            st.error(f"خطأ في خدمة التعرف على الكلام: {e}")
            return ""
        except Exception as e:
            st.error(f"خطأ في معالجة الصوت: {e}")
            return ""

    def generate_response(self, user_text):
        """Generate Arabic response using AI"""
        try:
            messages = [{"role": "system", "content": self.system_prompt}]

            # Add recent chat history
            for msg in st.session_state.chat_history[-5:]:
                if msg["user"]:
                    messages.append({"role": "user", "content": msg["user"]})
                messages.append({"role": "assistant", "content": msg["assistant"]})

            # Add current user message
            messages.append({"role": "user", "content": user_text})

            # Generate response
            response = self.openai_client.chat.completions.create(
                model="google/gemma-2-9b-it",
                messages=messages,
                max_tokens=200,
                temperature=0.7
            )

            return response.choices[0].message.content.strip()

        except Exception as e:
            return f"عذراً، حدث خطأ تقني: {str(e)[:50]}... يرجى المحاولة مرة أخرى."

    def generate_greeting(self):
        """Generate greeting message"""
        try:
            messages = [{"role": "system", "content": self.system_prompt}]
            messages.append({"role": "user", "content": "مرحباً، أود التحدث مع موظفة الاستقبال"})

            response = self.openai_client.chat.completions.create(
                model="google/gemma-2-9b-it",
                messages=messages,
                max_tokens=150,
                temperature=0.7
            )

            return response.choices[0].message.content.strip()

        except Exception as e:
            return "مرحباً، أهلاً وسهلاً بك في عيادة فانكوفر لطب الأسنان. اسمي ساندي، كيف يمكنني مساعدتك اليوم؟"

    def text_to_speech(self, text):
        """Convert text to speech using gTTS"""
        try:
            tts = gTTS(text=text, lang='ar', slow=False)
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
            tts.save(temp_file.name)
            return temp_file.name
        except Exception as e:
            st.error(f"خطأ في تحويل النص إلى صوت: {e}")
            return None

# Initialize assistant
@st.cache_resource
def get_assistant():
    return OnlineArabicVoiceAssistant()

def create_audio_recorder_html():
    """Create HTML5 audio recorder"""
    html_code = """
    <div style="text-align: center; padding: 20px; border: 2px dashed #ccc; border-radius: 10px;">
        <h3>🎙️ مسجل الصوت</h3>
        <div id="recorder-status">اضغط على "ابدأ التسجيل" للبدء</div>
        <br>
        <button id="start-btn" onclick="startRecording()"
                style="background-color: #ff4444; color: white; padding: 10px 20px; border: none; border-radius: 5px; margin: 5px;">
            🎙️ ابدأ التسجيل
        </button>
        <button id="stop-btn" onclick="stopRecording()" disabled
                style="background-color: #888; color: white; padding: 10px 20px; border: none; border-radius: 5px; margin: 5px;">
            ⏹️ إيقاف التسجيل
        </button>
        <button id="process-btn" onclick="processRecording()" disabled
                style="background-color: #4444ff; color: white; padding: 10px 20px; border: none; border-radius: 5px; margin: 5px;">
            🎯 معالجة التسجيل
        </button>
        <br><br>
        <audio id="audio-playback" controls style="display:none; width: 100%;"></audio>
        <div id="countdown" style="font-size: 24px; color: red; display: none;"></div>
    </div>

    <script>
    let mediaRecorder;
    let audioChunks = [];
    let recordingTimer;
    let audioBlob;

    async function startRecording() {
        try {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            mediaRecorder = new MediaRecorder(stream);

            mediaRecorder.ondataavailable = event => {
                audioChunks.push(event.data);
            };

            mediaRecorder.onstop = () => {
                audioBlob = new Blob(audioChunks, { type: 'audio/webm;codecs=opus' });
                const audioUrl = URL.createObjectURL(audioBlob);

                const audioElement = document.getElementById('audio-playback');
                audioElement.src = audioUrl;
                audioElement.style.display = 'block';

                document.getElementById('process-btn').disabled = false;
                document.getElementById('recorder-status').textContent = 'تم التسجيل! يمكنك الآن معالجته أو إعادة التسجيل';
            };

            mediaRecorder.start();
            audioChunks = [];

            document.getElementById('start-btn').disabled = true;
            document.getElementById('stop-btn').disabled = false;
            document.getElementById('process-btn').disabled = true;
            document.getElementById('recorder-status').textContent = 'جاري التسجيل... اضغط على "إيقاف" عند الانتهاء';

            // Auto stop after 10 seconds
            setTimeout(() => {
                if (mediaRecorder && mediaRecorder.state === 'recording') {
                    stopRecording();
                }
            }, 10000);

        } catch (error) {
            document.getElementById('recorder-status').textContent = 'خطأ: لا يمكن الوصول للمايك. تأكد من السماح للموقع بالوصول للمايك.';
        }
    }

    function stopRecording() {
        if (mediaRecorder && mediaRecorder.state === 'recording') {
            mediaRecorder.stop();
            mediaRecorder.stream.getTracks().forEach(track => track.stop());
        }

        document.getElementById('start-btn').disabled = false;
        document.getElementById('stop-btn').disabled = true;
    }

    function processRecording() {
        if (audioBlob) {
            const reader = new FileReader();
            reader.onloadend = () => {
                // Send audio data to Streamlit
                window.parent.postMessage({
                    type: 'audio-data',
                    data: reader.result
                }, '*');

                document.getElementById('recorder-status').textContent = 'تم إرسال التسجيل للمعالجة...';
            };
            reader.readAsDataURL(audioBlob);
        }
    }
    </script>
    """
    return html_code

def process_user_input(assistant, text, mode="text"):
    """Process user input and generate response"""
    if not text or not text.strip():
        st.warning("⚠️ لم يتم إدخال أي نص")
        return

    st.success(f"✅ **النص المستلم:** {text}")

    # Generate AI response
    with st.spinner("🤖 جاري إنشاء الرد..."):
        response = assistant.generate_response(text.strip())

    # Add to chat history
    st.session_state.chat_history.append({
        "user": text.strip(),
        "assistant": response,
        "mode": mode
    })

    # Generate audio response if needed
    if st.session_state.chat_mode in ["audio", "hybrid"]:
        with st.spinner("🔊 جاري تحويل الرد إلى صوت..."):
            audio_file = assistant.text_to_speech(response)

            if audio_file:
                # Store audio file reference
                msg_index = len(st.session_state.chat_history) - 1
                st.session_state[f"audio_{msg_index}"] = audio_file

    st.rerun()

def display_chat_history():
    """Display chat history with audio playback"""
    if not st.session_state.chat_history:
        st.info("💬 ابدأ محادثتك...")
        return

    for i, msg in enumerate(st.session_state.chat_history):
        # User message
        if msg["user"]:
            mode_emoji = "🎤" if msg.get("mode") == "audio" else "⌨️"
            st.markdown(f"**👤 المريض {mode_emoji}:** {msg['user']}")

        # Assistant response
        st.markdown(f"**🤖 ساندي:** {msg['assistant']}")

        # Audio playback if available
        audio_key = f"audio_{i}"
        if audio_key in st.session_state and os.path.exists(st.session_state[audio_key]):
            with open(st.session_state[audio_key], "rb") as audio_file:
                audio_bytes = audio_file.read()
            st.audio(audio_bytes, format="audio/mp3")

        st.divider()

def send_initial_greeting(assistant):
    """Send initial greeting from assistant"""
    if not st.session_state.first_message_sent:
        with st.spinner("🤖 مرحباً..."):
            greeting = assistant.generate_greeting()

            st.session_state.chat_history.append({
                "user": "",
                "assistant": greeting,
                "mode": "system"
            })

            # Generate audio greeting if needed
            if st.session_state.chat_mode in ["audio", "hybrid"]:
                audio_file = assistant.text_to_speech(greeting)
                if audio_file:
                    st.session_state["audio_0"] = audio_file

        st.session_state.first_message_sent = True
        st.rerun()

def main():
    """Main application"""

    # Title and header
    st.title("🦷 عيادة فانكوفر لطب الأسنان")
    st.markdown("### 🤖 مساعدة الاستقبال الذكية - ساندي (النسخة الأونلاين)")
    st.markdown("**مرحباً! يمكنك التحدث باللغة العربية نصياً أو صوتياً**")

    # Initialize assistant
    assistant = get_assistant()

    # Sidebar configuration
    with st.sidebar:
        st.header("⚙️ إعدادات النظام")

        # System status
        st.success("✅ النظام جاهز للعمل أونلاين")

        # Environment info
        if assistant.is_colab:
            st.info("🔬 تم اكتشاف بيئة Google Colab")
        elif not assistant.is_local:
            st.info("☁️ يعمل في بيئة السحابة")
        else:
            st.info("💻 يعمل محلياً")

        st.divider()

        # Chat mode selection
        st.header("💬 نمط المحادثة")
        chat_mode = st.selectbox(
            "اختر نمط التفاعل:",
            ["نصي", "صوتي", "مختلط"],
            index=2,
            help="النصي: كتابة فقط، الصوتي: تسجيل فقط، المختلط: كلاهما"
        )

        # Map selection to internal mode
        mode_map = {"نصي": "text", "صوتي": "audio", "مختلط": "hybrid"}
        st.session_state.chat_mode = mode_map[chat_mode]

        st.divider()

        # Clear chat
        if st.button("🗑️ مسح المحادثة"):
            st

2025-09-17 10:11:33.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:11:33.742 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:11:33.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:11:33.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:11:33.913 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:11:33.920 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:11:33.922 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 10:11:33.925 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [9]:
import streamlit as st

try:
    st.set_page_config(page_title="Demo")
except Exception as e:
    print("Streamlit context not available:", e)


2025-09-17 10:46:08.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
